*A quick note: You can run all of the code blocks below by hovering your mouse over the brackets at the top left of each block and clicking the play button that appears. In order to run a block of code, you have to have already run the subsequent blocks (In other words you have to run the code blocks in order)*

## **Creating A Neural Network Engine From Scratch**

**What is a Neural Network?**

The first thing you might be wondering is what a neural network is. I won't go into too much detail here but in essence, a **neural network** is an algorithm that is intented to simulate some of the features of the human brain that is used to interpret complex data and draw conclusions by making connections between different data points. If you want to learn more about the specifics of neural networks, you can chek out [this video](https://www.youtube.com/results?search_query=3blue1brown+neural+network).

**About This Python Notebook**

In this Python notebook, I will explain how I created my own object oriented neural network engine with no external dependencies (the only libraries I used were default Python libraries so no data science tools like numpy or pandas and no neural network libraries like tensorflow/keras, PyTorch, scikit, etc.). This tool can be used in a similar way to the traditionally used Python libraries for creating neural networks like tensorflow/keras. 

In order to gain a thorough fundamental understanding of exactly how neural networks work (especially the intuition and calculus behind backpropagation), I decided to creat this neural network framework from scratch to understand what is happening at every step of a neural network's process.

This will be more of a technical Python notebook because I can't split up the code into multiple code blocks (since I am using object oriented programming and the majority of the code is inside the NeuralNetwork object, I can't split up the code because all of the methods for the object need to be in one block). 

I will be posting a higher-level/fundamental explanation of everything going on here on [My Medium](https://medium.com/@mr.adam.maj) soon so make sure to check there if you are looking for a simpler explanation.

Since I cant split up the code, I will explain all of the code through in-code comments rather than text boxes. Look out for red text inside of triple quotes (""") or green text following hashtags (#) as these are the comments that I have left explaining the code.

# **Creating a Neural Network Class**

In the code below, we create a NeuralNetwork class using object orient programming. Instances of this class will be individual neural networks, with customizable layers. The idea is that the entire neural network will be fully customizable from the number of layers in the network to the number of neurons in each layer to the activation functions of each neuron.

**Run the code block below if you want to follow along when we create and train a neural network later.**

In [ ]:
from random import random
import math

class NeuralNetwork(object):
    """
    This class will create an instance of a neural network. The idea is that we 
    will be able to use the methods in this class to customize the neural network
    to the exact size that we want it to be (including the number of layers, number
    of neurons per layer, and the activations of the neurons). This means that we
    will need our forward and backpropgation algorithms to be programmed so that
    they will function properly no matter the size of the network.

    The code for this class is split into four main steps:
    1. Initialization: In this section, we enable the creation and customization 
       of the neural network (adding layers, changing activations, etc.)
    2. Forward Propagation: In this section, we create the forward propagation
       algorithm which will both store the outputs of each neuron during forward
       propagation and will return the output of the network (thus it can be used)
       when our network is fully trained to make predictions.
    3. Backward Propagation: In this section, we create the backward propagation
       algorithm which calculates the errors of each neuron in the network based
       on the error of the network calculated after forward propagation and adjusts
       the weights of each neuron accordingly.
    4. Training: In this section, we create the function which will enable us to
       train our network with specific training data. This is where we will be able 
       to specify relevant learning parameters like the learning rate and number of
       epochs that we want.
    """

    # Section 1: Initialization 
    def __init__(self):
        """
        When we first initialize the network, we don't need to do much. All we do
        here is create an empty list for our neural network (which can be filled)
        in later.
        """
        self.neural_network = list()
    
    def add_layer(self, num_neurons, activation = 'sigmoid', input_layer = False):
        """
        This is the function that allows the user to add layers to the neural 
        network. They can specify the number of neurons in the layer, the activation
        function used by the neurons in the layer (which is set by default to the
        sigmoid function but can be adjusted), and if the layer is the input layer.
        Note that the first layer added to the network must always be specified 
        as the input layer.
        """
        if not input_layer:
            #Each layer is represented as a list of neurons where each neuron
            #is represented as a dictionary with a bias, a set of weights corresponding
            #to the neurons in the previous layer, and an activation function.
            self.neural_network.append([{'bias': random(), 
                                         'weights': [random() for i in range(self.last_layer_neurons)],
                                         'activation': activation} 
                                         for i in range(num_neurons)])
        self.last_layer_neurons = num_neurons

    # Section 2: Forward Propagation
    def sigmoid(self, value, derivative = False):
        """
        This is the Sigmoid activation function which can be used in computing the
        activation of a neuron. It can be also used as the derivative of the sigmoid
        function (which you will see we need later in backpropagation).
        """
        if derivative:
            return value * (1 - value)
        return 1/(1 + math.exp(-1 * value))
    
    def relu(self, value, derivative = False):
        """
        This is the Rectified Linear Unit (ReLU) activation function which can be 
        used in computing the activation of a neuron. This function returns a value 
        of 0 if it is fed a value less than 0 and returns the value it was fed if 
        the value is greater than 0. It can also be used as the derivative of the 
        relu function (which again we will need in backpropagation).
        """
        if derivative:
            return 1 if value > 0 else 0
        return value if value > 0 else 0
    
    def activate(self, inputs, weights, bias, activation):
        """
        This function computes the activation of a specific neuron. It computes
        the dot product of the input and weight vectors (basically just multiplying
        all of the weights coming into the neuron with the corresponding outputs
        of the neurons they are coming from and then adding all of them up) and 
        then adds the bias to this number before passing the whole sum into the
        specified activation function.
        """
        activation_functions = {'sigmoid': self.sigmoid, 'relu': self.relu}
        activation_function = activation_functions[activation]
        
        return activation_function(sum([inputs[i] * weights[i] for i in range(len(inputs))]) + bias)
    
    def forward_propagate(self, inputs):
        """
        This function passes the specified inputs into the input layer of the neural
        network. From there, it computes the activation of each neuron in a layer 
        using the fuction above and then passes these activations to the neurons
        of the next layer, repeating this process until it reaches the output layer.
        """
        for layer in self.neural_network:
            layer_outputs = list()
            for neuron in layer:
                #Here we store the outputs of each neuron (computed using the activate
                #function we defined above) because we will need them for backpropagation
                neuron['output'] = self.activate(inputs, neuron['weights'], neuron['bias'], neuron['activation'])
                layer_outputs.append(neuron['output'])
            #Here is where we pass the outputs of one layer as the inputs of the next layer
            inputs = layer_outputs
        #Here we return the output of the final layer (which is the prediction of the network)
        return inputs
    
    # Section 3: Backward Propagation
    def backward_propagate(self, inputs, expected_outputs, learning_rate):
        """
        This function calculates the error of our network (the difference between
        the output predicted by the network and the output it was supposed to predict).
        In then uses this error of the last neurons in the network to calculate the 
        error of the neurons in the previous layer using some simple calculus. 
        Then, it adjust the weights and biases of our neural network based on the
        errors of each neuron (this is what actually makes the network more accurate).
        """
        #Here, we compute the errors of each neuron, focusing on the layers
        #In reverse order where we start by looking at the last layer
        for i in reversed(range(len(self.neural_network))):
            layer = self.neural_network[i]
            layer_errors = list()
            #If we are looking at the last layer...
            if i == len(self.neural_network) - 1:
                for j in range(len(layer)):
                    neuron = layer[j]
                    #Compute the error of the output neurons in our neural network
                    neuron_error = expected_outputs[j] - neuron['output']
                    layer_errors.append(neuron_error)
            else:
                #For any other layer besides the last layer...
                for j in range(len(layer)):
                    #Calculate the error associated with each neuron based on the 
                    #connected neurons in the following layer
                    neuron_error = sum([neuron['weights'][j] * neuron['error'] for neuron in self.neural_network[i + 1]])
                    layer_errors.append(neuron_error)
            for j in range(len(layer)):
                neuron = layer[j]
                #Finally, multiply the errors we found before by the derivative of 
                #activation function of each neuron (this comes from using the chain
                #rule from calculus).
                neuron['error'] = layer_errors[j] * self.sigmoid(neuron['output'], True)
        
        #Here, we adjust the weights of our network based on the errors we computed
        #This time we go through the neural network in order from the first to last layer
        for i in range(len(self.neural_network)):
            layer = self.neural_network[i]
            for neuron in layer:
                for j in range(len(neuron['weights'])):
                    #Here we adjust the weights based on the error associated with
                    #each neuron and the outputs of the previous layer. We also
                    #take into account the learning rate to specify how much we want
                    #to adjust our weights for each training example.
                    neuron['weights'][j] += learning_rate * inputs[j] * neuron['error']
                #Here we adjust the bias taking into account the learning rate
                neuron['bias'] += learning_rate * neuron['error']
            #Here we save the outputs of the current layer so that we can reference them
            #When adjusting the weights of the next layer.
            inputs = [neuron['output'] for neuron in self.neural_network[i]]
    
    #Section 4: Training
    def train(self, x_train, y_train, learning_rate, epochs):
        """
        Finally, we create the method which we wil use to access all of the above
        methods. This method will allow us to pass in training data into our neural
        network. The network will train with the specified number of epochs (iterations
        through the training data) and will use the specified learning rate.
        
        """
        for epoch in range(epochs):
            total_error = 0
            for i in range(len(x_train)):
                x = x_train[i]
                y = y_train[i]
                #Propagate our training inputs through our network
                outputs = self.forward_propagate(x)
                #Calculate the mean squared error of the predicted outputs (for display purposes)
                total_error += sum([(y[j] - outputs[j]) ** 2 for j in range(len(outputs))])
                #Adjust weights and biases with backpropagation
                self.backward_propagate(x, y, learning_rate)
            #Print a message to the console at the end of each epoch so we can see
            #The progress of the neural network/if it is improving
            print("Epoch: {}, Total Error: {:.3f}".format(epoch, total_error))

# **Building & Training A Simple Neural Network With The Engine**

In the code below, we create a neural network using our NeuralNetwork class that we created above. For the sake of simplicity, I will use a very basic sample dataset with a few datapoints (that way we can train our network quickly and see the results). However, note that this neural network engine is functional for much larger datasets as you can create as many layers and nodes as you want. If you want to see this neural network engine used in a more complex and applicable use case, check out [this Python notebook](https://colab.research.google.com/drive/1FWyLbDmi415bUFFmRhVFbLogtswRoHkd?usp=sharing) where I create an artificial neural network to analyze bank information using my neural network engine.

The dataset I will be using is from [this resource](https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/), which also goes over a higher level explanation of the code in this notebook (I adapted this code to make it object oriented, more readable, and simplified but also more powerful).

**Run the Code Below to Load In the Dataset**

In [ ]:
 #Each training example has two data points
 x_train = [[2.7810836, 2.550537003],
	          [1.465489372, 2.362125076],
	          [3.396561688, 4.400293529],
	          [1.38807019, 1.850220317],
	          [3.06407232, 3.005305973],
	          [7.627531214, 2.759262235],
	          [5.332441248, 2.088626775],
	          [6.922596716, 1.77106367],
	          [8.675418651, -0.242068655],
	          [7.673756466, 3.508563011]]
						
#Each of these outputs is meant to represent a binary value where [1, 0] represents
#0 and [0, 1] represents 1.
y_train = [[1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]]

Now that we have the simple dataset we will be using, let's create our neural network with the engine that we created. Since each training input has two values, we will create an input layer for our network with two neurons. Next, we will add a hidden layer with two neurons, and finally an output layer with two neurons, one to predict an output of 0 and one to predict an output of 1.

**Run the Code Below to Create Our Neural Network**

In [ ]:
#Create an instance of our NeuralNetwork class
model = NeuralNetwork()

#Add the input layer, hidden layer, and output layer, each with two neurons
model.add_layer(2, input_layer = True)
model.add_layer(2)
model.add_layer(2)

Now that we have our neural network, we can train the network and see it in action. Run the code below to train our model with the dataset we loaded in earlier. Notice how the error is decreasing as the neural networks is training and gets down to a very low number. That means that the neural network is working as intended!

**Run the Code Below to Train Our Neural Network**

In [ ]:
#Train our network with the datasets specified above. We are using a learning rate of 0.5
#And are training with 100 epochs.
model.train(x_train, y_train, 0.5, 100)

# **Thanks for Reading!**

My name is Adam Majmudar and I'm a 17 year old machine learning developer.

If you want to connect with me/see my work, check out the following links:

* [My Medium](https://medium.com/@mr.adam.maj)
* [My Linkedin](https://www.linkedin.com/in/adam-majmudar-24b596194/)
* [My Portfolio](https://tks.life/profile/adam.majmudar)


If you enjoyed this article, make sure to check out the following:

* [Creating a Neural Network to Analyze Data About Bank Customers With My Neural Network Engine](https://colab.research.google.com/drive/1FWyLbDmi415bUFFmRhVFbLogtswRoHkd?usp=sharing): In this notebook, I create a more advanced neural network with my neural network engine to showcase it being used to solve a real problem. 

* [How to Code a Neural Network With Backpropagation in Python](https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/): This notebook gives an excellent explanation of the concepts used in this code. I adapted the code in this article into an object oriented format so that it could be applied to a number of different problems. Additonally, I recreated the same concepts with my own code and different logic in specific areas to make the code more understandable and functional.